In [ ]:
'''
Copyright 2024 Michael Koefinger

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
'''

In [ ]:
'''
Author: Michael Koefinger
Date: 18.01.2024
Notebook to determine coefficients of a Delta-Sigma modulator using FIR feedback based on [1]

[1] S. Pavan, "Continuous-Time Delta-Sigma Modulator Design Using the Method of Moments," in IEEE Transactions on Circuits and Systems I: Regular Papers, vol. 61, no. 6, pp. 1629-1637, June 2014, doi: 10.1109/TCSI.2013.2290846.

'''

In [ ]:
import sympy as sym
from sympy import exp, factorial, binomial

In [ ]:
''' Calculate the l-th moment of a (DAC) pulse based on its Laplace transform P(s)
    
    P: Laplace transform of pulse
    s: Laplace variable
    l: l-th moment of the pulse
    
'''
def moment_lap(P,s,l):
    deriv = P
    for i in range(l):
        deriv = deriv.diff(s)
    res = sym.limit(deriv, s, 0)
    res = res*(-1)**l
    return res
    

In [ ]:
# Test moment_lap()
s = sym.Symbol('s')
n_fir = sym.symbols('n_fir')
P_nrz = (1-exp(-s))/s
P_rz = 2*(1-exp(-s/2))/s
P_nrz_fir = 1/n_fir*(1-exp(-n_fir*s))/s
for i in range(4):
    print(moment_lap(P_nrz,s,i))

for i in range(4):
    print(moment_lap(P_nrz_fir,s,i))

In [ ]:
''' Calculate the output of a chain of k intergrators given the Laplace transform of the input (DAC) pulse P(s)
    
    k: Number of integrators
    t: time variable
    P: Laplace transform of pulse
    s: Laplace variable
    
'''
def integ_chain_output(k, t, P, s):
    x_out = 0
    for l in range(k):
        x_out = x_out + ((-1)**l/(factorial(k-1))*binomial(k-1,l)*moment_lap(P,s,l)*t**(k-l-1))
    return x_out

In [ ]:
# Test integ_chain_output
t = sym.symbols('t')
for i in range(5):
    print(integ_chain_output(i,t,P_nrz,s))

In [ ]:
''' Calculate sampled sequence at the output of the chain of integrators, i.e. the loop filter of a Delta-Sigma modulator.

    a_list: List of coefficient names
    t: time variable
    N: Order of the loop filter, i.e. number of integrators
    P: Laplace transform of pulse
    s: Laplace variable

'''
def sampled_loopfilter_output(a_list, t, N, P, s):
    y_out = 0
    for k in range(1,N+1):
        a_k = a_list[k-1]
        x_k = integ_chain_output(k,t,P,s)
        y_out = y_out + a_k*x_k
        #print(k); print(a_k); print(x_k)
    return y_out

In [ ]:
# Use method of moments to find tuned loop filter coefficients for a FIR DAC
t = sym.symbols('t')
[a1, a2, a3, a4] = sym.symbols('a1 a2 a3 a4')
[a1_fir, a2_fir, a3_fir, a4_fir] = sym.symbols('a1_fir a2_fir a3_fir a4_fir')
a_list = [a1, a2, a3, a4]
a_list_fir = [a1_fir, a2_fir, a3_fir, a4_fir]
N = 4
out_samp_nrz = sampled_loopfilter_output(a_list, t, N, P_nrz,s)
out_samp_nrz_fir = sampled_loopfilter_output(a_list_fir, t, N, P_nrz_fir,s)
y_nrz = out_samp_nrz.as_poly(t)
y_nrz_fir = out_samp_nrz_fir.as_poly(t)
coeff_proto_list = y_nrz.all_coeffs()
coeff_fir_list = y_nrz_fir.all_coeffs()
eq_list = []
for i in range(N):
    eq_list.append(coeff_fir_list[i]-coeff_proto_list[i])
    
sol = sym.solve(eq_list, a_list_fir)
simplified_sol = sym.simplify(sol)

In [ ]:
sym.factor(simplified_sol)

In [ ]:
coeff_proto_list

In [ ]:
coeff_fir_list